In [1]:
import os
import numpy as np
import secrets
import pandas as pd
import scipy.stats as spstats
import statsmodels.api as sm
from joblib import Parallel, delayed
from tqdm import tqdm
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns

# Functions

In [30]:
def check_dir(dir: str):
  """
  Creates a given path directory "dir" if it does not exist.
  Args:
    dir (str): Path to the directory.
  """
  if os.path.exists(dir) and os.path.isdir(dir):
    pass
  else:
    os.makedirs(dir)


def spearman_corr(expression, mutation_burden):
  rho, pval = spstats.spearmanr(
    expression,
    mutation_burden,
  )
  return pd.Series({"rho": rho, "pval": pval,})


def driver_neighbour_corr(
  driver: int,
  neighbourlist: np.ndarray,
  mutationtab: np.ndarray,
  expressiontab: np.ndarray,
  ctfilter: np.ndarray | None = None,
) -> tuple:
  """
  Computes spearman rank correlation coefficient.

  Parameters:
  -----------
  driver: driver index to index in mutationtab
  
  neighbourlist: 1D (listlike) with neighbour indices to index expressiontab
  
  mutationtab: array with driver mutations of shape
      (# cancer types, # drivers)
      
  expressiontab: array with neighbour expression of shape
      (# cancer types, # neighbours)
      
  ctfilter: boolean array to remove cancer types without mutation in mutationtab
  and expressiontab with shape (# cancer types, # drivers)

  Returns:
  --------
  tuple with shape (rho list, p-value list, neighbour list)
  """
    
  if ctfilter is not None:
    filt = ctfilter[driver]
      
  else:
    filt = [True] * len(mutationtab)

  exp = expressiontab[np.ix_(filt, neighbourlist)]

  # remove neighbours without expression after cancer type filter
  mask = np.flatnonzero(exp.sum(axis=0) > 0)
    
  if len(mask) == 0:
    # if driver does not have neighbours with expression
    return [np.nan], [np.nan], [np.nan]
      
  elif len(mask) == 1:
    # if driver has only one neighbour with expression
    rho, pvalue = spstats.spearmanr(exp[:, mask], mutationtab[filt, driver])
    return [rho], [pvalue], neighbourlist[mask]
  
  else:
    rho, pvalue = spstats.spearmanr(exp[:, mask], mutationtab[filt, driver])
    return rho[-1, :-1], pvalue[-1, :-1], neighbourlist[mask]


def between_cancer_corr(
  exparray: np.ndarray,
  mutationarray: np.ndarray,
  graph: pd.DataFrame,
  cancertype_filter: np.ndarray | None = None,
  n_jobs: int = -1,
  progressbar: bool = True,
):
  """
  Computes Between Cancer Type Analysis
  
  Parameters:
  -----------
  exparray: array with neighbour expression of shape
      (# cancer types, # neighbours)
  
  graph: boolean array with shape (# neighbours, # drivers)
  
  mutationarray: array with driver mutations of shape
      (# cancer types, # drivers)
      
  cancertype_filter: boolean array to remove cancer types without mutation in
      mutationtab and expressiontab with shape (# cancer types, # drivers)

  Returns:
  --------
  pd.DataFrame with columns [driver, neighbour, rho, rho_pval]
  """
    
  driverlist = graph.columns.tolist()
  if progressbar:
    neighbourlist = tqdm([np.flatnonzero(graph[driver]) for driver in driverlist])
      
  else:
    neighbourlist = [np.flatnonzero(graph[driver]) for driver in driverlist]

  # compute correlation
  corr_results = Parallel(n_jobs=n_jobs)(
    delayed(driver_neighbour_corr)
    (driver, neighbours, mutationarray, exparray, cancertype_filter)
    for driver, neighbours in enumerate(neighbourlist)
  )

  results = {"driver": [], "neighbour": [], "rho": [], "rho_pval": [],}
  for driver in range(len(driverlist)):
    results["driver"].extend([driverlist[driver]] * len(corr_results[driver][2]))
    results["neighbour"].extend(corr_results[driver][2])
    results["rho"].extend(corr_results[driver][0])
    results["rho_pval"].extend(corr_results[driver][1])

  results = (
    pd.DataFrame(results)
    .set_index("neighbour")
    .merge(
      pd.Series(graph.index).rename("neighbour"),
      left_index=True,
      right_index=True,
    )
    .dropna()
    .reset_index(drop=True)
  )
  return results


def ols(y, x, index_x=False):
  """
  Ordinary Least Squares regression.
  Args:
      y (np.ndarray): Dependent variable.
      x (np.ndarray): Independent variable.
      index_x (bool): Set True if x is a DataFrame with the same index as y.
  Returns:
      tuple: OLS coefficients and residuals.
  """
  if index_x:
    x = x[y.name]
  filt = y>0
  x = sm.add_constant(x)
  model = sm.OLS(y[filt], x[filt]).fit()
  return model.resid

# Directories

In [3]:
rawdata = "data/raw/"
outputdata = "data/processed/"
shuffled_dir = outputdata+"shuffled_bct/"
check_dir(shuffled_dir)

# Mutation Burden

## Load Data

In [4]:
silent_mutation = ["Silent", "5'Flank", "3'Flank", "5'UTR", "3'UTR", "Intron", "RNA"]
mutationtab = (
    pd.read_table(rawdata+"mc3.v0.2.8.PUBLIC.xena",)
    [["sample", "gene", "effect"]]
    .assign(
      patient=lambda x: x["sample"].str.split("-").str[:-1].str.join("-"),
      nonsilent=lambda x: ~x["effect"].isin(silent_mutation))
    .drop(columns=["sample"])
    .drop_duplicates()
)
print(mutationtab.shape)
mutationtab.head()

(2625385, 4)


,gene,effect,patient,nonsilent
0,TACC2,Missense_Mutation,TCGA-02-0003,True
1,JAKMIP3,Silent,TCGA-02-0003,False
2,PANX3,Missense_Mutation,TCGA-02-0003,True
3,SPI1,Missense_Mutation,TCGA-02-0003,True
4,NAALAD2,Missense_Mutation,TCGA-02-0003,True


In [5]:
exptab = pd.read_feather(outputdata+"expression.feather")
clintab = exptab.index.to_frame(index=False)
print("# patients:", len(clintab))
print("# cancer types:", clintab.cancer_type.nunique())
display(exptab.head())
clintab.head()

# patients: 7317
# cancer types: 32


,,A1BG,A1CF,A2M,A2ML1,A4GALT,A4GNT,AAAS,AACS,AADAT,AAGAB,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
patient,cancer_type,,,,,,,,,,,,,,,,,,,,,
TCGA-02-0047,GBM,6.98,0.0,15.05,5.40,5.22,1.16,8.87,8.92,7.87,10.01,...,8.03,8.66,6.05,8.48,10.12,0.69,10.24,11.92,10.45,9.24
TCGA-02-0055,GBM,8.62,0.0,15.39,1.42,8.93,0.64,9.22,8.31,6.66,10.41,...,8.87,7.95,5.45,8.14,9.25,2.60,9.85,13.49,9.25,9.49
TCGA-02-2483,GBM,8.09,0.0,14.36,1.82,6.46,0.00,10.11,8.95,8.02,9.92,...,9.42,9.39,4.35,8.67,9.76,5.50,10.24,12.31,9.70,9.46
TCGA-02-2485,GBM,6.41,0.0,12.93,7.73,7.29,0.56,9.99,8.25,7.58,10.36,...,8.79,8.79,5.78,8.10,10.40,0.00,10.06,12.31,10.16,9.45
TCGA-02-2486,GBM,6.77,0.0,15.32,6.71,5.49,0.00,9.46,8.62,7.77,10.54,...,7.39,6.24,5.03,7.64,9.35,0.00,9.43,12.93,9.30,9.05


,patient,cancer_type
0,TCGA-02-0047,GBM
1,TCGA-02-0055,GBM
2,TCGA-02-2483,GBM
3,TCGA-02-2485,GBM
4,TCGA-02-2486,GBM


In [6]:
graph = pd.read_csv(outputdata+"main_graph.csv")
drivers = graph["driver"].unique()
neighbours = graph["neighbour"].unique()
print("# drivers:", len(drivers))
print("# neighbours:", len(neighbours))

# drivers: 3081
# neighbours: 15464


## Mutation Burden

In [ ]:
mutation_burden = (
    mutationtab
    .groupby("patient", as_index=False)
    ["nonsilent"]
    .sum()
    .merge(clintab, on="patient")
    .rename(columns={"nonsilent": "mutation_burden"})
    [["patient", "cancer_type", "mutation_burden"]]
)
print(mutation_burden.shape)
mutation_burden.to_csv(outputdata+"mutation_burden.csv", index=False)
mutation_burden.head()

## Driver-specific mutation burden

In [ ]:
mutation_burden = (
    mutationtab
    .merge(clintab, on="patient")
    .groupby(["cancer_type", "gene"], as_index=False)
    ["nonsilent"]
    # mean mutation burden per pair driver-cancer type
    .mean()
    .pivot(index="cancer_type", columns="gene", values="nonsilent")
    [drivers]
    .fillna(0)
    .reset_index()
)
print(mutation_burden.shape)
mutation_burden.to_csv(outputdata+"driver_specific_mutation_burden.csv", index=False)
mutation_burden.head()

## Correlation Between Mutation Burden and Gene Expression

In [ ]:
mutation_burden = pd.read_csv(outputdata+"mutation_burden.csv")
print(mutation_burden.shape)
mutation_burden.head()

In [ ]:
corrtab, mutation_burden = (
  exptab
  .align(mutation_burden.set_index(["patient", "cancer_type"]), join="inner", axis=0, level=0)
 )
print(corrtab.shape)
print(mutation_burden.shape)
display(corrtab.head())
mutation_burden.head()

In [ ]:
correlation = (
  corrtab
    .apply(
      lambda x: spearman_corr(
        x.to_numpy().flatten(),
      mutation_burden["mutation_burden"].to_numpy()
      ),
      axis=0,
    )
  .T
)
print(correlation.shape)
print(len(correlation.dropna()))
correlation.head()

In [ ]:
signcorr = correlation[correlation["pval"] < 0.05]
print(len(signcorr))
print("(+) correlation", len(signcorr[signcorr["rho"] > 0]))
print("(-) correlation", len(signcorr[signcorr["rho"] < 0]))

In [ ]:
correlation.to_csv(outputdata+"mutation_burden_corr.csv", index=False)

# Between Cancer Types Associations

## Load Data

In [7]:
graph = pd.read_feather(outputdata+"neighbours.feather")
print(graph.shape)
graph.head()

(15464, 3081)


driver,A1CF,A2ML1,ABCA10,ABCA13,ABCA7,ABCB1,ABCB5,ABCC3,ABCC5,ABCC9,...,ZRANB3,ZRSR2,ZSCAN31,ZSCAN4,ZSWIM3,ZSWIM6,ZSWIM7,ZWILCH,ZWINT,ZZEF1
neighbour,,,,,,,,,,,,,,,,,,,,,
A1BG,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
A1CF,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
A2M,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
A2ML1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
A4GALT,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [8]:
mutationtab = pd.read_feather(outputdata+"mutation.feather")
# number of individuals with mutations per cancer type
cancer_freq = (
  mutationtab.index.to_frame(index=False)
  .groupby("cancer_type", as_index=False)
  .size()
  .rename(columns={"size": "freq"})
)
# number of mutated individuals per cancer type
mutationtab = (
  mutationtab
  .groupby(["cancer_type"])
  .sum()
)

display(cancer_freq.head())
print(mutationtab.shape)
mutationtab.head()

,cancer_type,freq
0,ACC,73
1,BLCA,288
2,BRCA,760
3,CESC,253
4,CHOL,35


(32, 3081)


,A1CF,A2ML1,ABCA10,ABCA13,ABCA7,ABCB1,ABCB5,ABCC3,ABCC5,ABCC9,...,ZRANB3,ZRSR2,ZSCAN31,ZSCAN4,ZSWIM3,ZSWIM6,ZSWIM7,ZWILCH,ZWINT,ZZEF1
cancer_type,,,,,,,,,,,,,,,,,,,,,
ACC,0,0,0,1,0,0,1,1,0,1,...,2,1,0,0,0,1,0,0,0,0
BLCA,1,6,4,19,7,4,8,7,5,12,...,5,2,0,0,0,0,1,2,0,10
BRCA,1,5,6,15,7,4,5,4,6,10,...,2,0,0,0,4,2,0,3,2,4
CESC,3,4,1,10,3,1,3,3,2,5,...,2,1,0,1,0,0,0,2,1,2
CHOL,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
exptab = (
    pd.read_feather(outputdata+"expression.feather")
    .groupby(["cancer_type"])
    .mean()
)
print(exptab.shape)
exptab.head()

(32, 15464)


,A1BG,A1CF,A2M,A2ML1,A4GALT,A4GNT,AAAS,AACS,AADAT,AAGAB,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
cancer_type,,,,,,,,,,,,,,,,,,,,,
ACC,5.851781,0.135616,13.047671,3.620548,7.893425,0.310274,10.721507,10.201781,5.040000,10.077123,...,7.492740,8.361507,4.685342,7.888904,9.919589,1.240137,9.573425,10.610822,9.883151,8.889315
BLCA,5.422882,0.433264,12.652847,7.206562,9.460729,0.675278,9.714722,9.418819,7.122535,10.239965,...,8.883403,9.703507,4.867361,8.038889,9.965208,3.223333,9.590139,12.098681,9.826944,9.302361
BRCA,7.128882,0.109618,13.495566,3.458474,8.081000,0.671026,9.417474,9.965684,5.593118,10.629211,...,8.774500,9.377132,5.894934,8.994039,10.075250,6.206553,9.729579,11.800447,10.150566,9.807053
CESC,5.663557,0.382530,11.309130,8.673913,9.790198,0.755810,9.707510,10.105059,5.265534,10.303043,...,9.702885,10.714466,5.028024,8.102213,10.281028,6.270435,9.254150,11.797036,9.832688,9.449921
CHOL,7.678286,7.446000,12.957429,0.789429,8.179714,3.774571,9.802286,9.980000,5.859714,10.088000,...,8.234857,8.757429,5.628286,8.532286,10.021429,1.162857,9.445429,11.936571,10.002857,9.471714


In [10]:
mutation_burden = (
  pd.read_csv(outputdata+"mutation_burden.csv")
  .groupby("cancer_type", as_index=False)
  ["mutation_burden"]
  .mean()
  .merge(cancer_freq, on="cancer_type")
  .set_index("cancer_type")
  .sort_index()
  .rename(columns=lambda x: x.lower())
)
mutation_burden.head()

,mutation_burden,freq
cancer_type,,
ACC,34.712329,73
BLCA,119.857639,288
BRCA,46.130263,760
CESC,87.233202,253
CHOL,33.828571,35


In [11]:
specific_mutation_burden = pd.read_csv(
  outputdata+"driver_specific_mutation_burden.csv", index_col="cancer_type")
print(specific_mutation_burden.shape)
specific_mutation_burden.head()

(32, 3081)


,A1CF,A2ML1,ABCA10,ABCA13,ABCA7,ABCB1,ABCB5,ABCC3,ABCC5,ABCC9,...,ZRANB3,ZRSR2,ZSCAN31,ZSCAN4,ZSWIM3,ZSWIM6,ZSWIM7,ZWILCH,ZWINT,ZZEF1
cancer_type,,,,,,,,,,,,,,,,,,,,,
ACC,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,1.000000,0.500000,0.000000,0.500000,...,1.000000,1.0,0.0,0.0,0.0,1.000000,0.0,0.000000,0.0,0.000000
BLCA,0.333333,0.545455,0.666667,0.833333,0.727273,0.800000,0.888889,0.777778,0.625000,0.750000,...,1.000000,1.0,0.0,0.0,0.0,0.000000,1.0,1.000000,0.0,0.714286
BRCA,1.000000,0.555556,1.000000,0.625000,1.000000,0.500000,0.714286,0.571429,0.750000,0.769231,...,0.500000,0.0,0.0,0.0,0.8,0.666667,0.0,1.000000,1.0,1.000000
CESC,1.000000,0.666667,0.333333,0.625000,0.428571,0.333333,1.000000,0.428571,0.222222,0.454545,...,0.666667,0.5,0.0,0.5,0.0,0.000000,0.0,0.666667,1.0,0.666667
CHOL,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000


In [12]:
print("exptab:", exptab.shape)
print("mutationtab:", mutationtab.shape)
print("mutation_burden:", mutation_burden.shape)
print("specific_mutation_burden:", specific_mutation_burden.shape)
print("graph:", graph.shape)

exptab: (32, 15464)
mutationtab: (32, 3081)
mutation_burden: (32, 2)
specific_mutation_burden: (32, 3081)
graph: (15464, 3081)


## Spearman Correlation with real pairs

### Data preparation

In [14]:
exparray = (
    exptab
    .sort_index(level=0)
    .to_numpy()
)
mutationarray = (
    mutationtab
    # compute relative frequency
    .div(mutation_burden.freq, axis=0)
    # order axis
    .T.sort_index()
    .T.sort_index(level=0)
)
# filter to remove cancer types with no mutations prior to correlation calculation
cancertype_filter = (
  mutationarray.T
  .reset_index(names="driver")
  .set_index("driver")
  # remove cancer types with no mutations
  .transform(lambda x: x > 0)
)
cancertype_filter.to_csv(outputdata+"cancertype_filter.csv", index=True)
cancertype_filter = cancertype_filter.to_numpy()
print(cancertype_filter.T)

[[False False False ... False False False]
 [ True  True  True ...  True False  True]
 [ True  True  True ...  True  True  True]
 ...
 [False  True  True ... False  True  True]
 [False  True False ... False False False]
 [False False False ... False False False]]


In [15]:
normalization_dict = {
  "bct_corr": mutationarray.to_numpy(),
  "bct_corr_mb": mutationarray.div(mutation_burden.mutation_burden, axis=0).to_numpy(),
  "bct_corr_mb_specific": (mutationarray/specific_mutation_burden).fillna(0).to_numpy(),
  "bct_corr_mb_logspecific": (
    (mutationarray/np.log10(specific_mutation_burden+1))
    .fillna(0)
    .to_numpy()
  )
}
for key, value in normalization_dict.items():
    print(f"{key}: {value.shape}")

bct_corr: (32, 3081)
bct_corr_mb: (32, 3081)
bct_corr_mb_specific: (32, 3081)
bct_corr_mb_logspecific: (32, 3081)


In [16]:
print("exparray:", exparray.shape)
print("mutationarray:", mutationarray.shape)
print("specific_mutation_burden:", specific_mutation_burden.shape)
print("graph:", graph.shape)
print("cancertype_filter:", cancertype_filter.shape)

exparray: (32, 15464)
mutationarray: (32, 3081)
specific_mutation_burden: (32, 3081)
graph: (15464, 3081)
cancertype_filter: (3081, 32)


In [17]:
nmut = mutationtab.sum(axis=0).sort_index()
nct = pd.Series(cancertype_filter.sum(axis=1), index=nmut.index)
nct_nmut = pd.concat([nct, nmut], axis=1).rename(columns={0: "nct", 1: "nmut"})
nct_nmut.head()

,nct,nmut
A1CF,16,39
A2ML1,21,67
ABCA10,20,73
ABCA13,28,266
ABCA7,20,66


### Compute Correlation: all cancer types

In [ ]:
for norm, mutarray in tqdm(normalization_dict.items()):
    print(norm)
    # compute correlation
    corr = between_cancer_corr(
        exparray,
        mutarray,
        graph,
        cancertype_filter,
    )
    # save results
    (
      corr
      .merge(nct_nmut, left_on="driver", right_index=True)
      .to_csv(outputdata+f"{norm}.csv", index=False)
    )

### Compute Correlation: OLS

In [ ]:
mutarray = (
  mutationarray
  .apply(ols, axis=0, args=(mutation_burden.mutation_burden,))
  .fillna(0)
  .to_numpy()
)
corr = between_cancer_corr(
        exparray,
        mutarray,
        graph,
        cancertype_filter,
    )
corr.head()

In [ ]:
mutarray = (
  mutationarray
  .apply(ols, axis=0, args=(specific_mutation_burden, True))
  .fillna(0)
  .to_numpy()
)
spec_corr = between_cancer_corr(
        exparray,
        mutarray,
        graph,
        cancertype_filter,
    )
spec_corr.head()

In [ ]:
filtgraph = pd.read_csv(outputdata+"main_graph_filtered.csv")
pd.merge(corr, filtgraph).to_csv(outputdata+"bct_corr_mb_ols.csv", index=False)
pd.merge(spec_corr, filtgraph).to_csv(outputdata+"bct_corr_mb_specific_ols.csv", index=False)

### Compute Correlation: low and high mutation frequency groups

In [ ]:
mutarray = normalization_dict["bct_corr_mb_specific"]
median = np.nanmedian(np.where(mutarray>0, mutarray, np.nan), axis=0)
low_mut_filter = cancertype_filter & (mutarray <= median).T
high_mut_filter = cancertype_filter & (mutarray >= median).T
driver_indexer = np.flatnonzero(high_mut_filter.sum(axis=1)>2)

mutarray = mutarray[:, driver_indexer]
low_mut_group = low_mut_filter[driver_indexer]
high_mut_group = high_mut_filter[driver_indexer]
filtgraph = graph.iloc[:, driver_indexer]
print("mutarray:", mutarray.shape)
print("low_mut_group:", low_mut_group.shape)
print("high_mut_group:", high_mut_group.shape)
print("filtgraph:", filtgraph.shape)

In [ ]:
corr_results = []
for group, filter in zip(["low_mut", "high_mut"], [low_mut_group, high_mut_group]):
    print(group)
    # compute correlation
    corr = between_cancer_corr(
        exparray,
        mutarray,
        filtgraph,
        filter,
    )
    # save results
    corr_results.append(corr)

corr_results = pd.merge(
    *corr_results,
    on=["driver", "neighbour"],
    suffixes=("_low_mut", "_high_mut"),
)

In [ ]:
corr_results.to_csv(outputdata+"bct_corr_mutation_groups.csv", index=False)

## Spearman Correlation with fake pairs

### Data preparation

In [ ]:
fakepairs = (
  pd.read_csv(outputdata+"full_pathway_analysis.csv", usecols=["driver", "fake_neighbour"])
  .dropna()
)
drivers = fakepairs.driver.unique()
fake_neighbours = fakepairs.fake_neighbour.unique()
print("# drivers:", len(drivers))
print("# fake neighbours:", len(fake_neighbours))

In [ ]:
exparray = (
    exptab[fake_neighbours]
    .sort_index(level=0)
    .to_numpy()
)
mutationarray = (
    mutationtab[drivers]
    # compute relative frequency
    .div(mutation_burden.freq, axis=0)
    # order axis
    .T.sort_index()
    .T.sort_index(level=0)
)
# filter to remove cancer types with no mutations prior to correlation calculation
cancertype_filter = (
  mutationarray[drivers].T
  .reset_index(names="driver")
  .set_index("driver")
  # remove cancer types with no mutations
  .transform(lambda x: x > 0)
  .to_numpy()
)
mutationarray = (mutationarray/specific_mutation_burden[drivers]).fillna(0).to_numpy()

In [ ]:
fake_graph = (
  fakepairs
  .assign(value=1)
  .pivot_table(index="fake_neighbour", columns="driver", values="value", fill_value=0)
  .astype(bool)
)
fake_graph.head()

In [ ]:
print("exparray:", exparray.shape)
print("mutationarray:", mutationarray.shape)
print("fake graph:", fake_graph.shape)
print("cancertype_filter:", cancertype_filter.shape)

### Compute Correlation

In [ ]:
# compute correlation
corr = between_cancer_corr(
  exparray,
  mutationarray,
  fake_graph,
  cancertype_filter,
)
# save results
corr.to_csv(outputdata+"bct_corr_fake_pairs.csv", index=False)

# Filtered BCT Pairs

In [ ]:
maingraph = pd.read_csv(outputdata+"main_graph.csv")
pairs_to_exclude = pd.read_csv(outputdata+"pairs_to_exclude_bct.csv")
print("# pairs in main graph:", len(maingraph))
print("# pairs to exclude:", len(pairs_to_exclude))
display(maingraph.head())
pairs_to_exclude.head()

In [ ]:
filteredpairs = (
  maingraph
  .merge(pairs_to_exclude, how="outer", indicator=True)
  .query("_merge == 'left_only'")
  .drop(columns=["_merge"])
)
print("pairs in filtered graph:", len(filteredpairs))
print("# drivers:", filteredpairs.driver.nunique())
print("# neighbours:", filteredpairs.neighbour.nunique())

In [ ]:
bct_corr = pd.read_csv(outputdata+"bct_corr_mb_specific.csv")
print(len(bct_corr))
bct_corr.head()

In [ ]:
filtered_bct_corr = (
  bct_corr.merge(filteredpairs)
  [["driver", "neighbour", "rho", "rho_pval", "nct", "nmut"]]
)
print("# pairs:", len(filtered_bct_corr))
print("# significant pairs:", len(filtered_bct_corr[filtered_bct_corr.rho_pval < 0.05]))
print("# drivers:", filtered_bct_corr.driver.nunique())
print("# neighbours:", filtered_bct_corr.neighbour.nunique())
filtered_bct_corr.head()

In [ ]:
filteredpairs.to_csv(outputdata+"main_graph_filtered.csv", index=False)
filtered_bct_corr.to_csv(outputdata+"bct_corr_filtered.csv", index=False)

# Random shuffles

## Load Data

In [ ]:
shufflepairs = pd.read_csv(outputdata+"main_graph_filtered.csv")
drivers = shufflepairs.driver.unique()
neighbours = shufflepairs.neighbour.unique()
print("# shuffle pairs:", len(shufflepairs))
print("# drivers:", len(drivers))
print("# neighbours:", len(neighbours))
shufflepairs.head()

In [ ]:
exptab = (
  pd.read_feather(outputdata+"expression.feather")
  [neighbours]
  # drop patient labels
  .droplevel(0)
)
print(exptab.shape)
exptab.head()

In [ ]:
mutationtab = pd.read_feather(outputdata+"mutation.feather")

cancer_freq = (
  mutationtab.index.to_frame(index=False)
  .groupby("cancer_type")
  .size()
)

mutationtab = (
  mutationtab[drivers]
  .groupby(["cancer_type"])
  .sum()
  .div(cancer_freq, axis=0)
  # order axis
  .T.sort_index()
  .T.sort_index()
)
print(cancer_freq.shape)
print(mutationtab.shape)
mutationtab.head()

In [ ]:
mutation_burden = (
    pd.read_csv(outputdata+"mutation_burden.csv")
    .groupby("cancer_type", as_index=False)
    ["mutation_burden"]
    .mean()
    .set_index("cancer_type")
    .sort_index()
    .mutation_burden
)
specific_mutation_burden = (
  pd.read_csv(
    outputdata+"driver_specific_mutation_burden.csv", index_col="cancer_type")
  [drivers]
)
display(mutation_burden.head(2))
specific_mutation_burden.head(2)

In [ ]:
print("exptab:", exptab.shape)
print("mutationtab:", mutationtab.shape)
print("mutation_burden:", mutation_burden.shape)
print("specific_mutation_burden:", specific_mutation_burden.shape)
print("graph:", graph.shape)

## Data preparation

In [ ]:
# filter to remove cancer types with no mutations prior to correlation calculation
cancertype_filter = (
  mutationtab.T
  .reset_index(names="driver")
  .set_index("driver")
  # remove cancer types with no mutations
  .transform(lambda x: x > 0)
  .to_numpy()
)

In [ ]:
shuffle_graph = (
  shufflepairs
  .assign(value=1)
  .pivot_table(index="neighbour", columns="driver", values="value", fill_value=0)
  .astype(bool)
)
shuffle_graph.head()

## shuffle

In [ ]:
mutationarray = (mutationtab/specific_mutation_burden[drivers]).fillna(0).to_numpy()
print("mutationarray:", mutationarray.shape)
print("shuffle graph:", shuffle_graph.shape)
print("cancertype_filter:", cancertype_filter.shape)

In [ ]:
print(secrets.randbits(128))

In [ ]:
seed = 61028791052787875347939602848649680021
rng = np.random.default_rng(seed)
spawn = rng.spawn(100)
index = exptab.index.to_numpy()

for i, child_rng in enumerate(tqdm(spawn)):
  # shuffle expression data
  shuffled_index = pd.Index(child_rng.permutation(index), name="cancer_type")
  exparray = (
    exptab
    .set_index(shuffled_index)
    .groupby(["cancer_type"])
    .mean()
    .sort_index()
    .to_numpy()
  )

  # compute correlation
  corr = between_cancer_corr(
    exparray,
    mutationarray,
    shuffle_graph,
    cancertype_filter,
    progressbar=False,
  )
  # save results
  corr.to_feather(shuffled_dir+f"shuffle{i}.feather")

## Shuffle OLS

In [ ]:
mutationarrays = dict(
   mb = (
    mutationtab
    .apply(ols, axis=0, args=(mutation_burden,))
    .fillna(0)
    .to_numpy()
  ),
   mb_specific = (
    mutationtab
    .apply(ols, axis=0, args=(specific_mutation_burden, True))
    .fillna(0)
    .to_numpy()
  ),
)
print("mutationarray mb:", mutationarrays["mb"].shape)
print("mutationarray mb_specific:", mutationarrays["mb_specific"].shape)
print("shuffle graph:", shuffle_graph.shape)
print("cancertype_filter:", cancertype_filter.shape)

In [ ]:
print(secrets.randbits(128))
check_dir(outputdata+"shuffle_bct_mb_ols/")
check_dir(outputdata+"shuffle_bct_mb_specific_ols/")

In [ ]:
seed = 234739320750563714933441023972028553133
rng = np.random.default_rng(seed)
spawn = rng.spawn(100)
index = exptab.index.to_numpy()

for i, child_rng in enumerate(tqdm(spawn)):
  # shuffle expression data
  shuffled_index = pd.Index(child_rng.permutation(index), name="cancer_type")
  exparray = (
    exptab
    .set_index(shuffled_index)
    .groupby(["cancer_type"])
    .mean()
    .sort_index()
    .to_numpy()
  )
  for norm, mutationarray in mutationarrays.items():
    # compute correlation
    corr = between_cancer_corr(
      exparray,
      mutationarray,
      shuffle_graph,
      cancertype_filter,
      progressbar=False,
    )
    # save results
    corr.to_feather(outputdata+f"shuffle_bct_{norm}_ols/shuffle{i+1}.feather")

# Cancer-specific driver BCT

In [ ]:
ncg_drivers = pd.read_table(
  rawdata+"NCG_cancerdrivers_annotation_supporting_evidence.tsv"
)
print(ncg_drivers.shape)
ncg_drivers.info()

## Export data for cancer type matching

In [ ]:
ncg_cancertypes = (
  ncg_drivers
  .cancer_type
  .dropna()
  .drop_duplicates()
  .reset_index(drop=True)
)
print("# cancer types:", len(ncg_cancertypes))
ncg_cancertypes.to_excel(outputdata+"ncg_cancertypes.ods", index=False)
ncg_cancertypes.head()

## Load Data

In [ ]:
exptab = (
  pd.read_feather(outputdata+"expression.feather")
  .groupby(["cancer_type"])
  .mean()
  .sort_index()
)
print(exptab.shape)
exptab.head()

In [ ]:
match = (
  pd.read_excel(outputdata+"cancer_type_matching.ods", sheet_name="NCG_mapping")
  .dropna()
  .set_index("NCG_cancer_type")
  .merge(
    ncg_drivers[["symbol", "cancer_type"]],
    left_index=True,
    right_on="cancer_type",
    how="inner",
  )
  .drop_duplicates()
)
match.to_csv(outputdata+"cancer_type_matching.csv", index=False)
print("# NCG cancer types:", match.cancer_type.nunique())
print("# TCGA cancer types:", match.TCGA_cancer_type.nunique())
print("# NCG drivers:", match.symbol.nunique())

match = (
  match
  .assign(value=1)
  .pivot_table(
    index="TCGA_cancer_type",
    columns="symbol",
    values="value",
    fill_value=0,
  )
  # remove cancer types not in exptab
  .loc[exptab.index, :]
  # keep only drivers with 3 or more mutations
  #.loc[:, lambda x: x.sum() > 2]
  .astype(bool)
)
print(match.shape)
match.head()

In [ ]:
maingraph = pd.read_csv(outputdata+"main_graph_filtered.csv")
drivers = np.intersect1d(maingraph.driver.unique(), match.columns)

# filter main graph to only include drivers in NCG
maingraph = maingraph[maingraph.driver.isin(drivers)]
drivers = maingraph.driver.unique()
neighbours = maingraph.neighbour.unique()
print("# shuffle pairs:", len(maingraph))
print("# drivers:", len(drivers))
print("# neighbours:", len(neighbours))
maingraph.head()

In [ ]:
# refilter match and exptab
exptab = exptab[neighbours]
match = match[drivers]

In [ ]:
mutationtab = pd.read_feather(outputdata+"mutation.feather")

cancer_freq = (
  mutationtab.index.to_frame(index=False)
  .groupby("cancer_type")
  .size()
)

mutationtab = (
  mutationtab[drivers]
  .groupby(["cancer_type"])
  .sum()
  .div(cancer_freq, axis=0)
  # order axis
  .T.sort_index()
  .T.sort_index()
)
print(cancer_freq.shape)
print(mutationtab.shape)
mutationtab.head()

In [ ]:
mutation_burden = (
  pd.read_csv(
    outputdata+"driver_specific_mutation_burden.csv", index_col="cancer_type")
  [drivers]
)
print(mutation_burden.shape)
mutation_burden.head()

In [ ]:
print("match:", match.shape)
print("exptab:", exptab.shape)
print("mutationtab:", mutationtab.shape)
print("mutation_burden:", mutation_burden.shape)

## Data preparation

In [ ]:
# filter to keep only cancer types where the driver has mutations
# and is annotated in NCG as a cancer driver in the given cancer types
cancertype_filter = (
  mutationtab.T
  .reset_index(names="driver")
  .set_index("driver")
  # remove cancer types with no mutations
  .transform(lambda x: x > 0)
)
# keep only cancer types where the driver is annotated in NCG
cancertype_filter = (
  (cancertype_filter & match.T)
  # remove drivers with no mutations in more than 2 cancer types
  .loc[lambda x: x.sum(axis=1) > 2]
)
# drivers that passed all filters
drivers = cancertype_filter.index.to_numpy()
print(cancertype_filter.shape)
nct = cancertype_filter.sum(axis=1)
cancertype_filter.to_csv(outputdata+"cancertype_filter_ncg.csv", index=True)
cancertype_filter.head()

In [ ]:
# refilter dataframes
maingraph = maingraph[maingraph.driver.isin(drivers)]
# neighbours that passed all filters
neighbours = maingraph.neighbour.unique()
# create arrays
exparray = exptab[neighbours].to_numpy()
mutationarray = (mutationtab[drivers]/mutation_burden[drivers]).fillna(0).to_numpy()
cancertype_filter = cancertype_filter.to_numpy()

graph = (
  maingraph
  .assign(value=1)
  .pivot_table(index="neighbour", columns="driver", values="value", fill_value=0)
  .astype(bool)
)
graph.head()

In [ ]:
print("exparray:", exparray.shape)
print("mutationarray:", mutationarray.shape)
print("graph:", graph.shape)
print("cancertype_filter:", cancertype_filter.shape)

## Compute Correlation

In [ ]:
# compute correlation
corr = between_cancer_corr(
  exparray,
  mutationarray,
  graph,
  cancertype_filter,
  progressbar=True,
).merge(nct.rename("nct"), left_on="driver", right_index=True)
# save results
corr.to_csv(outputdata+"bct_corr_ncg_drivers.csv", index=False)

# Results Analysis

## BCT correlation with mutation frequency groups

In [18]:
filted_bct_corr = pd.read_csv(outputdata+"bct_corr_filtered.csv")
grouped_bct_corr = pd.read_csv(outputdata+"bct_corr_mutation_groups.csv")
display(filted_bct_corr.head(2))
grouped_bct_corr.head(2)

,driver,rho,rho_pval,neighbour,nct,nmut
0,A1CF,-0.011765,0.965508,APOB,16,39
1,A1CF,0.282353,0.289350,APOBEC1,16,39


,driver,rho_low_mut,rho_pval_low_mut,neighbour,rho_high_mut,rho_pval_high_mut
0,A1CF,0.404762,0.319889,APOB,0.452381,0.260405
1,A1CF,-0.047619,0.910849,APOBEC1,0.047619,0.910849


In [19]:
results = pd.merge(
  filted_bct_corr,
  grouped_bct_corr,
  on=["driver", "neighbour"],
)[[
  "driver", "neighbour", "rho", "rho_low_mut",
  "rho_high_mut", "rho_pval", "rho_pval_low_mut", "rho_pval_high_mut"
  ]]

In [20]:
print("# pairs:", len(results))
print("# significant pairs:", (results.rho_pval < 0.05).sum())
print("# significant pairs (low mutation):", (results.rho_pval_low_mut < 0.05).sum())
print("# significant pairs (high mutation):", (results.rho_pval_high_mut < 0.05).sum())
print("# significant pairs in common:",
      len(results[
          (results.rho_pval < 0.05) &
          (results.rho_pval_low_mut < 0.05) &
          (results.rho_pval_high_mut < 0.05)
          ]))
stats = []
for pair_type in ["All pairs", "Significant pairs", "Non-significant pairs"]:
  print()
  print(pair_type)
  if pair_type == "All pairs":
    data = results
  elif pair_type == "Significant pairs":
    data = results[results.rho_pval < 0.05]
  else:
    data = results[results.rho_pval >= 0.05]

  concordance = data.assign(
    rho=lambda x: x.rho/x.rho.abs(),
    rho_low_mut=lambda x: x.rho_low_mut/x.rho_low_mut.abs(),
    rho_high_mut=lambda x: x.rho_high_mut/x.rho_high_mut.abs(),
  )

  for comparison in [
    ("rho", "rho_low_mut"),
    ("rho", "rho_high_mut"),
    ("rho_low_mut", "rho_high_mut"),
  ]:
    rho, pval = spstats.spearmanr(
      data[comparison[0]], data[comparison[1]], alternative="two-sided")
    signconc = (
      (concordance[comparison[0]] == concordance[comparison[1]]).sum()
      /len(concordance)* 100
    ).round(1)
    stats.append({
      "pair_type": pair_type,
      "comparison": f"{comparison[0]} vs {comparison[1]}",
      "rho": round(rho, 3),
      "pval": round(pval, 3),
      "sign_concordance_pct": signconc,
    })
pd.DataFrame(stats).set_index(["pair_type", "comparison"])

# pairs: 319863
# significant pairs: 39421
# significant pairs (low mutation): 23848
# significant pairs (high mutation): 21378
# significant pairs in common: 452

All pairs

Significant pairs

Non-significant pairs


rho  pval  \
pair_type             comparison                                 
All pairs             rho vs rho_low_mut           0.455   0.0   
                      rho vs rho_high_mut          0.401   0.0   
                      rho_low_mut vs rho_high_mut -0.030   0.0   
Significant pairs     rho vs rho_low_mut           0.677   0.0   
                      rho vs rho_high_mut          0.652   0.0   
                      rho_low_mut vs rho_high_mut  0.410   0.0   
Non-significant pairs rho vs rho_low_mut           0.377   0.0   
                      rho vs rho_high_mut          0.324   0.0   
                      rho_low_mut vs rho_high_mut -0.117   0.0   

                                                   sign_concordance_pct  
pair_type             comparison                                         
All pairs             rho vs rho_low_mut                           65.5  
                      rho vs rho_high_mut                          63.3  
                      rho_low_mut vs rho_high_mut                  47.5  
Significant pairs     rho vs rho_low_mut                           85.2  
                      rho vs rho_high_mut                          82.3  
                      rho_low_mut vs rho_high_mut                  70.6  
Non-significant pairs rho vs rho_low_mut                           62.7  
                      rho vs rho_high_mut                          60.6  
                      rho_low_mut vs rho_high_mut                  44.3

## Cancer-specific driver BCT

In [21]:
print("# NCG pairs:", len(pd.read_csv(outputdata+"bct_corr_ncg_drivers.csv")))
results = (
  pd.read_csv(outputdata+"bct_corr_filtered.csv")
  .merge(
    pd.read_csv(outputdata+"bct_corr_ncg_drivers.csv"),
    on=["driver", "neighbour"],
    suffixes=("", "_ngc"),
  )
  [[
    "driver", "neighbour", "rho", "rho_ngc", "rho_pval",
    "rho_pval_ngc", "nct", "nct_ngc"
  ]]
)
sign_intersection = results[
  (results.rho_pval < 0.05) & (results.rho_pval_ngc < 0.05)
]
results.head()

# NCG pairs: 47399


,driver,neighbour,rho,rho_ngc,rho_pval,rho_pval_ngc,nct,nct_ngc
0,ABCC9,ABCA2,-0.542727,-0.5,0.006139,0.666667,24,3
1,ABCC9,ABCB6,-0.018265,0.5,0.932493,0.666667,24,3
2,ABCC9,ABCC8,-0.554468,1.0,0.004928,0.000000,24,3
3,ABCC9,ABCD4,-0.301370,0.5,0.152398,0.666667,24,3
4,ABCC9,ABCG5,0.028267,0.5,0.895686,0.666667,24,3


In [ ]:
print("# pairs in common:", len(results))
print("# significant pairs (normal):", (results.rho_pval < 0.05).sum())
print("# significant pairs (NCG):", (results.rho_pval_ngc < 0.05).sum())
print("# significant pairs in common:", len(sign_intersection))
print("significant jaccard:", len(sign_intersection) / (
  (results.rho_pval < 0.05).sum() + (results.rho_pval_ngc < 0.05).sum()
))
rho, pval = spstats.spearmanr(
  results.rho, results.rho_ngc, alternative="two-sided"
)
print(f"Spearman correlation: rho={rho}, p-value={pval}")

In [ ]:
results.nct_ngc.describe()

In [ ]:
ctmatching = pd.read_csv(outputdata+"cancer_type_matching.csv")
ctmatching.head()

In [ ]:
ctstats = (
  ctmatching
  .groupby("symbol", as_index=False)
  .nunique()
  .rename(
    columns={"cancer_type": "NCG_cancer_type"}
  )
  [lambda x: x.symbol.isin(results.driver.unique())]
  .reset_index(drop=True)
)
display(ctstats.head(2))
ctstats.describe()

In [ ]:
data = results[results.driver.isin(
  ctstats.loc[ctstats.NCG_cancer_type > 7, "symbol"].unique()
)]
print(len(data))
print(data.driver.nunique())
spstats.spearmanr(
  data.rho, data.rho_ngc, alternative="two-sided"
)

In [ ]:
x = results[["driver", "nct", "nct_ngc"]].drop_duplicates().set_index("driver")
display(x.describe())
(x.nct/x.nct_ngc).rename("ratio").describe()

In [ ]:
ctfilter_ncg = pd.read_csv(outputdata+"cancertype_filter_ncg.csv", index_col="driver")
ctfilter = (
  pd.read_csv(outputdata+"cancertype_filter.csv", index_col="driver")
  .loc[ctfilter_ncg.index]
)
mutationtab = (
  pd.read_feather(outputdata+"mutation.feather")
  [ctfilter.index]
)
mutationtab = (
  mutationtab
  .groupby(["cancer_type"])
  .sum()
  # order axis
  .T.sort_index()
  .sort_index(axis=1)
)
print(ctfilter_ncg.shape, ctfilter.shape, mutationtab.shape)
display((
 mutationtab[ctfilter_ncg].mean(axis=1)/mutationtab[ctfilter].mean(axis=1)
).describe())
spstats.mannwhitneyu(
  mutationtab[ctfilter_ncg].mean(axis=1), 
  mutationtab[ctfilter].mean(axis=1),
  alternative="greater"
)

## BCT Pathway Analysis

In [22]:
pathway_analysis = pd.read_csv(outputdata+"full_pathway_analysis.csv")
print(len(pathway_analysis))
pathway_analysis.head()

324638


,driver,neighbour,fake_neighbour,jaccard,mean_pagerank_fake,median_pagerank_fake,mean_pagerank_real,median_pagerank_real,distance,reg_TFs,n_shortest_paths
0,ABCA7,ADRB2,TOR1AIP1,1.000000,0.000748,0.000748,0.000748,0.000748,5.0,"SPI1,TP53",2.0
1,ABCA7,APOA1,ATP2B1,0.375000,0.000113,0.000058,0.000111,0.000066,5.0,"CEBPA,E2F4,EGR1,ESR1,GATA6,HNF4A,PPARA,PPARG,R...",10.0
2,ABCA7,HNRNPD,E2F8,0.600000,0.000100,0.000147,0.000278,0.000149,5.0,"E2F4,ETS1,MITF,MYC",4.0
3,ABCA7,LLGL2,PTK6,0.545455,0.000172,0.000049,0.000089,0.000042,5.0,"ESR1,SOX9",2.0
4,ABCA7,SNX27,SLC5A11,0.666667,0.000073,0.000073,0.000064,0.000063,5.0,"CEBPA,HNF4A",2.0


In [23]:
associations = pd.read_csv(outputdata+"bct_corr_mb_specific.csv")
associations.head()

,driver,rho,rho_pval,neighbour,nct,nmut
0,A1CF,-0.011765,0.965508,APOB,16,39
1,A1CF,0.282353,0.289350,APOBEC1,16,39
2,A1CF,0.297059,0.263863,APOBEC2,16,39
3,A1CF,0.270588,0.310761,APOBEC3A,16,39
4,A1CF,0.247059,0.356275,APOBEC3B,16,39


In [24]:
fake_associations = pd.read_csv(outputdata+"bct_corr_fake_pairs.csv")
fake_associations.head()

,driver,rho,rho_pval,neighbour
0,ABCA7,0.127820,0.591246,ATP2B1
1,ABCA7,-0.153383,0.518524,CSNK1G2
2,ABCA7,-0.236090,0.316294,E2F8
3,ABCA7,-0.093233,0.695829,MEAF6
4,ABCA7,-0.359398,0.119633,MKNK1


In [25]:
tf_neighbour_pairs = pathway_analysis.loc[
  lambda x: x.distance == 1, ["driver", "neighbour", "fake_neighbour"]
]
print(len(tf_neighbour_pairs))

3283


In [26]:
print("# associations:", len(associations))
print("# associations (p-value < 0.05):",
      len(associations[lambda x: x["rho_pval"] < 0.05]))
print("proportion of significant associations:",
      len(associations[lambda x: x["rho_pval"] < 0.05]) / len(associations))
print("proportion of significant associations with TFs:",
      len(associations[lambda x: x["rho_pval"] < 0.05]
          .merge(tf_neighbour_pairs, on=["driver", "neighbour"])) /
      len(tf_neighbour_pairs))
print("# fake associations:", len(fake_associations))
print("# fake associations (p-value < 0.05):",
      len(fake_associations[lambda x: x["rho_pval"] < 0.05]))
print("proportion of significant fake associations:",
      len(fake_associations[lambda x: x["rho_pval"] < 0.05]) / len(fake_associations))

# associations: 455441
# associations (p-value < 0.05): 54460
proportion of significant associations: 0.11957641055592272
proportion of significant associations with TFs: 0.15016752969844654
# fake associations: 262578
# fake associations (p-value < 0.05): 30204
proportion of significant fake associations: 0.11502867719306263


In [27]:
pathway_associations = (
  associations
  .merge(pathway_analysis, on=["driver", "neighbour"], how="inner")
  .assign(sign=lambda x: x["rho_pval"] < 0.05)
  [[
    "driver", "neighbour", "mean_pagerank_real", "median_pagerank_real", "sign"
  ]]
  .melt(
    id_vars=["driver", "neighbour", "sign"],
    var_name="pagerank_type",
    value_name="pagerank_value"
  )
  .assign(
    pagerank_type=lambda x: x["pagerank_type"].str.split("_").str[0],
  )
  .assign(pair_type="real")
  .drop_duplicates()
)
print(len(pathway_analysis))
print(len(pathway_associations)/2)
pathway_associations.head()

324638
324565.0


,driver,neighbour,sign,pagerank_type,pagerank_value,pair_type
0,ABCA7,ADRB2,False,mean,0.000748,real
1,ABCA7,APOA1,False,mean,0.000111,real
2,ABCA7,HNRNPD,False,mean,0.000278,real
3,ABCA7,LLGL2,False,mean,0.000089,real
4,ABCA7,SNX27,True,mean,0.000064,real


In [28]:
fake_pathway_associations = (
  fake_associations
  .merge(
    pathway_analysis,
    left_on=["driver", "neighbour"],
    right_on=["driver", "fake_neighbour"],
    how="inner",
    suffixes=("_fake", "")
  )
  .drop(columns=["neighbour_fake", "neighbour"])
  .assign(sign=lambda x: x["rho_pval"] < 0.05)
  [[
    "driver", "fake_neighbour", "mean_pagerank_fake", "median_pagerank_fake", "sign"
  ]]
  .drop_duplicates()
  .melt(
    id_vars=["driver", "fake_neighbour", "sign"],
    var_name="pagerank_type",
    value_name="pagerank_value"
  )
  .assign(
    pagerank_type=lambda x: x["pagerank_type"].str.split("_").str[0],
  )
  .rename(columns={"fake_neighbour": "neighbour"})
  .assign(pair_type="fake")
)
print(len(pathway_analysis))
print(len(fake_pathway_associations)/2)
fake_pathway_associations.head()

324638
262578.0


,driver,neighbour,sign,pagerank_type,pagerank_value,pair_type
0,ABCA7,ATP2B1,False,mean,0.000113,fake
1,ABCA7,CSNK1G2,False,mean,0.000130,fake
2,ABCA7,E2F8,False,mean,0.000100,fake
3,ABCA7,MEAF6,False,mean,0.000049,fake
4,ABCA7,MKNK1,False,mean,0.000501,fake


In [ ]:
all_associations = (
  pd.concat([pathway_associations, fake_pathway_associations], ignore_index=True)
)
print(len(all_associations))
all_associations.head()

In [ ]:
quantiles = (
  all_associations
  .groupby(["pair_type", "pagerank_type"])
  ["pagerank_value"]
  .quantile(np.arange(0.1, 1.1, 0.1))
  .reset_index(name="quantile_value")
  .rename(columns={"level_2": "quantile"})
)
quantiles[lambda x: x["quantile"]==0.5].head()

In [ ]:
groups = (
  all_associations
  .merge(quantiles, on=["pair_type", "pagerank_type"])
  [lambda x: x["pagerank_value"] <= x["quantile_value"]]
  .groupby(
    [
      "driver", "neighbour", "pair_type", "pagerank_type", "pagerank_value",
      "sign"
    ],
    as_index=False
  )
  ["quantile"].min()
)
print(all_associations.shape[0])
groups.info()

In [ ]:
sign = (
  groups[groups["sign"]]
  .groupby(["pair_type", "pagerank_type", "quantile"], as_index=False)
  .size()
  .pivot(
    index=["pair_type", "pagerank_type",],
    columns="quantile",
    values="size"
  )
)
all = (
  groups
  .groupby(["pair_type", "pagerank_type", "quantile"], as_index=False)
  .size()
  .pivot(
    index=["pair_type", "pagerank_type",],
    columns="quantile",
    values="size"
  )
)
(sign/all).T

In [ ]:
mean = (
  pd.qcut(
    (
      pathway_associations
      [lambda x: x.pagerank_type=="mean"]
      .drop(columns=["pagerank_type", "pair_type"])
      .set_index(["driver", "neighbour"])
      ["pagerank_value"]
    ),
    q=10, duplicates="drop"
  )
  .reset_index(name="quantile")
  .merge(pathway_associations[["driver", "neighbour", "sign"]].drop_duplicates(), on=["driver", "neighbour"])
  .groupby(["quantile", "sign"], as_index=False)
  .size()
  .set_index("quantile")
)
mean.loc[mean.sign, "size"]/mean.groupby("quantile")["size"].sum()